In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
GEN_BACK_TR = True

GEN_UPSAMPLE = False

GEN_EN_ONLY = False

In [ ]:
#python basics
from matplotlib import pyplot as plt
import math, os, re, time
import numpy as np, pandas as pd, seaborn as sns
from multiprocessing import Pool
#nlp augmentation
!pip install --quiet googletrans==3.1.0a0 --quiet
from googletrans import Translator

#model evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold

#for fast parallel processing
from dask import bag, diagnostics

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.1 MB/s eta 0:00:00


In [ ]:
# Read input CSV file as DataFrame
df = pd.read_csv('/content/drive/MyDrive/NLP/data/clean-hindi-train.csv')
df = df[df.text.notnull()]
df['text'] = df['text'].str[:100]
df.head()

,label,text
0,1,मेट्रो की इस लाइन के चलने से दक्षिणी दिल्ली से...
1,1,प्रतीक खुलेपन का आज़ाद ख्याली का और भीड़ से अल...
2,1,ख़ासकर पिछले 10 साल तक प्रधानमंत्री रहे मनमोहन...
3,1,चीनी भाषा में ‘नीहाव’ (क्या हाल हैं) कहकर वो अ...
4,1,मुंबई पुलिस की क्राईम ब्रांच को इस जांच का जिम...


In [ ]:
def back_translate(sequence,label):
    languages = ['en', 'te','kn', 'es']
    #instantiate translator
    translator = Translator()
    output_sequence = [sequence]
    label_sequence = [label]
    #store original language so we can convert back
    org_lang = 'hi'
    #randomly choose language to translate sequence to  
    for language  in languages:
      #translate to new language and back to original
      translated = translator.translate(sequence, dest = language).text
      #translate back to original language
      translated_back = translator.translate(translated, dest = org_lang).text
      if translated_back not in output_sequence:
          output_sequence.append(translated_back)
          label_sequence.append(label)
    
    
    news_list.extend(output_sequence)
    label_list.extend(label_sequence)


#check performance
# output = back_translate('ನಾನು ಜಾಲಿ ಜೀವನ ನಡೆಸುತ್ತಿದ್ದೇನೆ')
# print(output)

In [ ]:
def back_translate_parallel(dataset):
    news = (dataset['text']).tolist()
    labels = (dataset['label']).tolist()
    with diagnostics.ProgressBar():
        for i in range(0,len(news)):
          print(i)
          back_translate(news[i],labels[i])

In [ ]:
news_list = list()
label_list=list()
if GEN_BACK_TR:
#now we apply translation augmentation
    back_translate_parallel(df)
    dic = {"text":news_list,"label":label_list}
    df = pd.DataFrame(dic)
    df.to_csv("BackTranslationAugData.csv",index=False)